# Primjer korištenja alata AutoGen


## Uvezi potrebne pakete


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Kreiranje klijenta

U ovom primjeru koristit ćemo [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) za pristup LLM-u.

`model` je definiran kao `gpt-4o-mini`. Pokušajte promijeniti model u neki drugi dostupan na GitHub Models marketplaceu kako biste vidjeli različite rezultate.

Kao brzi test, jednostavno ćemo pokrenuti jednostavan upit - `Koji je glavni grad Francuske`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Definiranje funkcija

U ovom primjeru, agentu ćemo omogućiti pristup alatu koji je funkcija s popisom dostupnih destinacija za odmor i njihovom dostupnošću.

Možete zamisliti da bi ovo bio scenarij u kojem putnički agent ima pristup, na primjer, bazi podataka o putovanjima.

Dok prolazite kroz ovaj primjer, slobodno pokušajte definirati nove funkcije i alate koje agent može pozvati.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Definiranje alata funkcije  
Kako bi agent koristio `vacation_destinations` kao `FunctionTool`, potrebno ga je definirati kao takvog.  

Također ćemo pružiti opis alata koji pomaže agentu da prepozna za što se taj alat koristi u odnosu na zadatak koji je korisnik zatražio.


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Definiranje agenta

Sada možemo kreirati agenta u donjem kodu. Definiramo `system_message` kako bismo agentu dali upute o tome kako pomoći korisnicima u pronalaženju destinacija za odmor.

Također postavljamo parametar `reflect_on_tool_use` na true. Ovo omogućuje LLM-u da uzme odgovor alata i pošalje ga koristeći prirodni jezik.

Možete postaviti parametar na false kako biste vidjeli razliku.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Pokretanje agenta

Sada možemo pokrenuti agenta s početnom korisničkom porukom u kojoj se traži putovanje u Tokio.

Možete promijeniti ovu destinaciju grada kako biste vidjeli kako agent reagira na dostupnost grada.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati autoritativnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane čovjeka. Ne preuzimamo odgovornost za nesporazume ili pogrešna tumačenja koja mogu proizaći iz korištenja ovog prijevoda.
